In [3]:
from imutils import face_utils
import matplotlib.pyplot as plt
import cv2
import dlib
import pywt
import os
import math
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.layers import Conv2D,MaxPooling2D, GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import Activation,Dense,Flatten
from keras.models import Sequential,load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.utils import to_categorical

Using TensorFlow backend.


In [4]:
train_folder = '/home/vishwajeet/Documents/Summer_Project_FER/Viola-Jones_jaffe/dataset1/'
preprocessing_folder = '/home/vishwajeet/Documents/Summer_Project_FER/Viola-Jones_jaffe/images_with_neutral/'
expres_code = ['NE','HA','AN','DI','FE','SA','SU']
expressions = [ 0,   1,   2,   3,   4,   5,   6]

In [11]:
face_detector =cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
def preprocessing(dir_path):
    img_list = os.listdir(dir_path)
    count=0
    for img in img_list:
        input_img = cv2.imread(dir_path + img)
        input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        label = img[3:5]  # each name of image have 2 char for label from index 3-5
#         gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(input_img, 1.3, 5)
        for (x,y,w,h) in faces:
            cv2.rectangle(input_img, (x,y), (x+w,y+h), (0,0,0), 2)     
            count += 1
            face_roi=input_img[y:y+h,x:x+w]
            face_roi = cv2.resize(face_roi,(128,128))
        # Save the captured image into the datasets folder
            cv2.imwrite("/home/vishwajeet/Documents/Summer_Project_FER/Viola-Jones_jaffe/dataset1/img." + str(label) + '.' +  
                    str(count) + ".jpg",face_roi )
        

In [12]:
preprocessing(preprocessing_folder)

In [13]:
def read_data(dir_path):
    img_names = []
    img_data_list = []
    labels = []
    img_list = os.listdir(dir_path)
    for img in img_list:
        input_img = cv2.imread(dir_path + img, cv2.IMREAD_GRAYSCALE)
        img_data_list.append(input_img)
        label = img[4:6]  # each name of image have 2 char for label from index 3-5
        labels.append(expres_code.index(label))
        img_names.append(img)
    img_data = np.array(img_data_list)
    print(img_data)
    return img_data, labels, img_names


In [14]:
X, Y, img_names = read_data(train_folder)
img_x=np.array(X)
img_x.shape

[[[  0   1   0 ...   0   1   0]
  [  2  29  63 ...  68  65  10]
  [  0  56 131 ... 142 139  23]
  ...
  [  0  42  87 ... 129 130  16]
  [  0  39  83 ... 132 128  22]
  [  2   5  13 ...  22  21   4]]

 [[  2   0   0 ...   2   0   0]
  [  0   7  26 ...  37  44   6]
  [  4  26  65 ... 110 111  11]
  ...
  [  0  18  53 ... 116 113  13]
  [  0  20  48 ... 115 111  13]
  [  1   2   6 ...  14  11   1]]

 [[  1   0   0 ...   0   0   0]
  [  0  16  55 ...   0   0   0]
  [  0  55 165 ...   2   1   0]
  ...
  [  0  35 103 ...   4   4   0]
  [  0  44 103 ...   4   4   2]
  [  1   1   9 ...   0   1   0]]

 ...

 [[  0   0   1 ...   1   0   0]
  [  0   3   7 ...   4  14   1]
  [  0   6  11 ...  12  33   6]
  ...
  [  0  20  79 ... 141 145  11]
  [  0  24  80 ... 146 145  14]
  [  1   0   8 ...  11  13   0]]

 [[  0   0   0 ...   0   1   1]
  [  3  19  46 ...  45  40   3]
  [  0  48 117 ... 102 104  20]
  ...
  [  0   0   2 ...   3   3   0]
  [  0   1   2 ...   3   3   0]
  [  0   1   1 ...   0   1  

(213, 128, 128)

In [15]:
labels_encoding = pd.get_dummies(Y, sparse=True)
labelenc= np.asarray(labels_encoding)
# print(labelenc.shape)
# print(labelenc[:43])
x_train, x_test, y_train, y_test = train_test_split(img_x, 
                                                    labelenc, 
                                                    test_size=0.2, 
                                                    stratify=np.array(labelenc), 
                                                    random_state=4)


In [17]:
rows=128
cols=128
x_train=x_train.reshape(x_train.shape[0],rows,cols,1)
x_test=x_test.reshape(x_test.shape[0],rows,cols,1)

input_shape=(rows,cols,1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0
x_train.shape,x_test.shape

((170, 128, 128, 1), (43, 128, 128, 1))

In [18]:
model = Sequential()

model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128,kernel_size=(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=256,kernel_size=(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(7))
model.add(Activation('softmax'))


In [19]:
model.compile(loss = "categorical_crossentropy",optimizer = Adam(), metrics=['accuracy'],)
BATCH_SIZE=85
epochs=50

model.fit(x_train, y_train,epochs=epochs, batch_size=BATCH_SIZE,verbose=1)

model.save('jaffe_model_viola_jones.h5')

Epoch 1/50
170/170 [==============================] - 5s 31ms/step - loss: 2.0372 - accuracy: 0.1765
Epoch 2/50
170/170 [==============================] - 4s 24ms/step - loss: 1.9973 - accuracy: 0.1706
Epoch 3/50
170/170 [==============================] - 4s 25ms/step - loss: 1.9441 - accuracy: 0.1471
Epoch 4/50
170/170 [==============================] - 4s 24ms/step - loss: 1.9270 - accuracy: 0.2471
Epoch 5/50
170/170 [==============================] - 4s 24ms/step - loss: 1.9069 - accuracy: 0.2471
Epoch 6/50
170/170 [==============================] - 4s 26ms/step - loss: 1.8828 - accuracy: 0.2588
Epoch 7/50
170/170 [==============================] - 5s 27ms/step - loss: 1.8034 - accuracy: 0.3471
Epoch 8/50
170/170 [==============================] - 5s 28ms/step - loss: 1.6708 - accuracy: 0.4294
Epoch 9/50
170/170 [==============================] - 5s 28ms/step - loss: 1.4454 - accuracy: 0.5118
Epoch 10/50
170/170 [==============================] - 5s 28ms/step - loss: 1.4243 - accura

In [20]:
model = load_model("jaffe_model_viola_jones.h5")

model.compile(loss = "categorical_crossentropy",optimizer = Adam(), metrics=['accuracy'],)

loss, acc = model.evaluate(x_test, y_test)
print('ACCURACY: ', acc)

43/43 [==============================] - 1s 23ms/step
ACCURACY:  0.8372092843055725


In [21]:
y_prediction = model.predict_classes(x_test)
y_test_original=np.argmax(y_test,axis=1)
confusion=confusion_matrix(y_true=y_test_original, y_pred=y_prediction)
print(confusion)

[[5 0 0 0 0 1 0]
 [0 6 0 0 0 0 0]
 [0 0 3 2 0 1 0]
 [0 0 0 5 1 0 0]
 [1 0 0 0 6 0 0]
 [0 0 0 1 0 5 0]
 [0 0 0 0 0 0 6]]


In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 64)      640       
_________________________________________________________________
activation_1 (Activation)    (None, 126, 126, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 128)       73856     
_________________________________________________________________
activation_2 (Activation)    (None, 61, 61, 128)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 256)      